<a href="https://www.kaggle.com/code/robertgrantham/austin-crime-reports-analysis?scriptVersionId=143236599" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Analyzing Austin PD's Crime Reports Dataset

The Austin Police Department's Crime Reports dataset provides valuable information about reported crimes in Austin, Texas. In this analysis, we will explore the dataset and gain insights into various aspects of crime in Austin.

## Table of Contents

I. Introduction  
II. Data  
III. Methodology  
IV. Outcome  

## I. Introduction

The Austin Police Department's Crime Reports dataset offers a comprehensive view of reported crimes in Austin, Texas. By analyzing this dataset, we can extract valuable insights into the nature and patterns of crime in the city.

## II. Data

### 1. Project Context

This project is self-paced and not affiliated with any work or educational institution. My goal is to uncover valuable insights that can benefit the Austin law enforcement community, news outlets, and anyone interested in understanding and addressing crime-related issues in the Austin area.

### 2. Data Collection

Initially, I attempted to import the data into this notebook using Sodapy's Socrata API method. However, I found it to be insufficient as it didn't import the complete dataset and added redundant columns. As a result, I decided to manually download the entire dataset and repeat the process each week after the updates.

### 3. Data Scrubbing

Before conducting any analysis, it is crucial to clean and preprocess the dataset. The data scrubbing process involves addressing missing data, removing irrelevant columns, and ensuring data types are appropriate.

## III. Methodology

Once the dataset is scrubbed and prepared, we can perform an exploratory analysis to gain a deeper understanding of the crime data. The exploratory analysis may include the following:

### A. Crime Rates by Area

#### 1. Crime Rates by Zip Code

We start by examining the crime rates by zip code. The following code calculates the total number of crimes for each zip code and displays the top 10 zip codes with the highest crime rates.

```python
crime_by_zipcode = df["zip_code"].value_counts().head(10).to_frame()
display(crime_by_zipcode)
```

#### 2. Crime Rates as Percentages

In addition to the total crime counts, it is also informative to analyze the crime rates as percentages relative to the total. The following code calculates the crime rates as percentages for each zip code and displays the top 10 zip codes with the highest crime rates.

```python
crime_rates_by_zipcode = df["zip_code"].value_counts(normalize=True).head(10).to_frame()
display(crime_rates_by_zipcode)
```

#### 3. Visualizing Crime Rates by Zip Code

To visually represent the crime rates by zip code, we can create a bar chart. The following code generates a bar chart showing the top 10 zip codes with the highest crime rates.

```python
plt.figure(figsize=(10, 6))
crime_by_zipcode.plot(kind="bar", rot=45)
plt.title("Top 10 Zip Codes with Highest Crime Rates")
plt.xlabel("Zip Code")
plt.ylabel("Number of Crimes")
plt.show()
```

## IV. Outcome

In this section, we summarize the key findings and insights obtained from the analysis. Discuss any significant patterns, trends, or observations related to crime in Austin based on the data and methodologies applied.

By analyzing the crime data, we aim to contribute to a better understanding of crime distribution, trends, and areas of concern in Austin. This information can be valuable for law enforcement agencies, policymakers, and the community in efforts to address and prevent crime effectively.

In [ ]:
# importing necessary libraries and configurations
import folium
from folium import plugins

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import itertools
import warnings
import os
from statsmodels.tsa.seasonal import seasonal_decompose

pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")
plt.style.use("seaborn-white")
sb.set_style("whitegrid")
%matplotlib inline

In [ ]:
# loading the data
df = pd.read_csv("/kaggle/input/crime-reports-09122023/Crime_Reports.csv")

In [ ]:
# examining the dataframe
display(df.info())
display(df.isnull().sum())
display(df.head())
display(df.tail())

In [ ]:
def clean_data(df):
    """
    Clean the crime data by removing unnecessary columns, renaming columns, filling missing values,
    converting data types, and creating additional time-based features.

    Args:
        df (pandas.DataFrame): Crime data.

    Returns:
        pandas.DataFrame: Cleaned crime data.
    """
    drop_col = [
        "Highest Offense Code",
        "Incident Number",
        "Occurred Time",
        "Occurred Date Time",
        "Report Date",
        "Report Time",
        "UCR Category",
        "X-coordinate",
        "Y-coordinate",
        "Location",
    ]
    df.drop(drop_col, axis=1, inplace=True)
    df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)
    
    fillna_cols = [
        "zip_code",
        "location_type",
        "council_district",
        "pra",
        "census_tract",
        "location_type",
        "apd_district",
        "apd_sector",
        "clearance_status",
        "category_description",
    ]
    df[fillna_cols].fillna("Unknown", inplace=True)

    date_cols = ["occurred_date", "report_date_time", "clearance_date"]
    cat_cols = [
        "highest_offense_description",
        "zip_code",
        "location_type",
        "council_district",
        "apd_district",
        "apd_sector",
        "pra",
        "census_tract",
        "category_description",
    ]

    df.family_violence.replace({"Y": "True", "N": "False"}, inplace=True)
    df.clearance_status.replace({"C": "True", "O": "True", "N": "False"}, inplace=True)

    df[cat_cols] = df[cat_cols].astype("category")
    df[date_cols] = df[date_cols].astype("datetime64")

    df["year"] = pd.to_datetime(df["occurred_date"], format="%m/%d/%Y").dt.year
    df["month"] = pd.to_datetime(df["occurred_date"], format="%m/%d/%Y").dt.month
    df["week"] = pd.to_datetime(df["occurred_date"], format="%m/%d/%Y").dt.week
    df["day"] = pd.to_datetime(df["occurred_date"], format="%m/%d/%Y").dt.day
    # df["hour"] = pd.to_datetime(df["occurred_date"], format="%m/%d/%Y").dt.hour

    df.set_index(["occurred_date"], inplace=True)
    df.sort_index(inplace=True)
    
    return df

df = clean_data(df)

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
# Re-examining the dataframe
display(df.info())
display(df.head())
display(df.tail())

In [ ]:
# Set colorblind-friendly palette
sb.set_palette("colorblind")

# Crime Rates by Zip Code
# Displaying dataframes for crime rates by zip code
crime_counts = df["zip_code"].value_counts().head(25).to_frame()
crime_percentages = df["zip_code"].value_counts(normalize=True).head(25).to_frame()

# Display the number of crimes by zip code
display(crime_counts)

# Display the percentage of total crimes by zip code
display(crime_percentages)

# Visualizing the top 25 crime-ridden zip codes in Austin
plt.figure(figsize=(8, 4), dpi=100)
top_zip_codes = df["zip_code"].value_counts().head(25)
top_zip_codes.plot.bar(rot=60)
plt.title("Overall Crime by the Top 25 Zip Codes (2003-present)")
plt.xlabel("Zip Code")
plt.ylabel("Number of Crimes")
plt.savefig("top_zip_codes.png")  # Save the figure
plt.show()

# Calculate the percentage of total crimes for the top 25 zip codes
top_zip_code_percentage = (top_zip_codes / len(df)) * 100

# Plot the percentage of total crimes by zip code
plt.figure(figsize=(12, 6))
top_zip_code_percentage.plot(kind="bar", rot=60)
plt.title("Percentage of Total Crimes by Top 25 Zip Codes")
plt.xlabel("Zip Code")
plt.ylabel("Percentage of Total Crimes")
plt.savefig("percentage_of_total_crimes.png")  # Save the figure
plt.show()

# Creating a time series plot with a 12-month rolling average
plt.figure(figsize=(12, 6), dpi=100)
monthly_crimes = df.resample("M").size()
monthly_crimes.plot(color="red", linewidth=1.5, label="Total per month")
monthly_crimes.rolling(window=12).mean().plot(
    color="orange", linewidth=2, label="12-month Moving Average"
)
plt.title("Monthly Crimes (2003-present)")
plt.xlabel("Month")
plt.ylabel("Number of Crimes")
plt.legend()
plt.savefig("monthly_crimes.png")  # Save the figure
plt.show()

# Decompose the time series
decomposition = seasonal_decompose(monthly_crimes, model="additive")

# Plot decomposed components
plt.figure(figsize=(12, 12))
plt.subplot(411)
decomposition.trend.plot(ax=plt.gca())
plt.title("Trend")
plt.subplot(412)
decomposition.seasonal.plot(ax=plt.gca())
plt.title("Seasonal")
plt.subplot(413)
decomposition.resid.plot(ax=plt.gca())
plt.title("Residual")
plt.subplot(414)
decomposition.observed.plot(ax=plt.gca())
plt.title("Observed")
plt.tight_layout()
plt.savefig("seasonal_decomposition.png")  # Save the figure
plt.show()

# Get the decomposed components
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Calculate summary statistics for each component
trend_summary = trend.describe()
seasonal_summary = seasonal.describe()
residual_summary = residual.describe()

# Print the summaries
print("Trend Component Summary:")
print(trend_summary)

print("\nSeasonal Component Summary:")
print(seasonal_summary)

print("\nResidual Component Summary:")
print(residual_summary)



### How is crime distributed in 78701 (downtown Austin)?

In [ ]:
# Filtering data for the 78701 area
df_01 = df[df['zip_code'] == 78701]

# Create a dataframe for the top crime categories in the zipcode
df_01_off = df_01['highest_offense_description'].value_counts().head(24)

# Display the count of different crime categories
display(df_01_off.to_frame())

# Display the crime categories as percentages
display(df_01['highest_offense_description'].value_counts(normalize=True).head(24).to_frame())

# Plotting a pie chart for crime distribution in 78701
plt.figure(figsize=(8, 8), dpi=100)
df_01_off.plot.pie(title="Crime Distribution (78701)")
plt.ylabel('')  # Remove y-label
plt.show()

### To analyze other zip codes, simply update the 'zip_code' variable accordingly.


### How are violent crimes distributed?

In [ ]:
# Creating separate dataframes for violent crime & murder rates
df_viol = df.query(
    'category_description == ["Aggravated Assault", "Robbery", "Rape", "Murder"]'
)
df_mur = df[df.category_description == "Murder"]
df_agg_asslt = df[df.category_description == "Aggravated Assault"]
df_robbery = df[df.category_description == "Robbery"]
df_rape = df[df.category_description == "Rape"]

# Creating yearly dataframes
# Annual overall crime
df_17 = df[df.year == 2017]
df_18 = df[df.year == 2018]
df_19 = df[df.year == 2019]
df_20 = df[df.year == 2020]
df_21 = df[df.year == 2021]
df_22 = df[df.year == 2022]


# Annual violent crime
df_viol_17 = df_viol[df_viol.year == 2017]
df_viol_18 = df_viol[df_viol.year == 2018]
df_viol_19 = df_viol[df_viol.year == 2019]
df_viol_20 = df_viol[df_viol.year == 2020]
df_viol_21 = df_viol[df_viol.year == 2021]
df_viol_22 = df_viol[df_viol.year == 2022]

# Annual murders
df_mur_17 = df_mur[df_mur.year == 2017]
df_mur_18 = df_mur[df_mur.year == 2018]
df_mur_19 = df_mur[df_mur.year == 2019]
df_mur_20 = df_mur[df_mur.year == 2020]
df_mur_21 = df_mur[df_mur.year == 2021]
df_mur_22 = df_mur[df_mur.year == 2022]

In [ ]:
# Filtering data for violent crimes
df_viol = df.query(
    'category_description == ["Aggravated Assault", "Robbery", "Rape", "Murder"]'
)
df_mur = df.query('category_description == "Murder"')
df_agg_asslt = df.query('category_description == "Aggravated Assault"')
df_robbery = df.query('category_description == "Robbery"')
df_rape = df.query('category_description == "Rape"')

# Creating yearly dataframes for overall crime
dfs_overall = [df[df.year == year] for year in range(2017, 2022)]

# Creating yearly dataframes for violent crime
dfs_violent = [df_viol[df_viol.year == year] for year in range(2017, 2022)]

# Creating yearly dataframes for murders
dfs_murders = [df_mur[df_mur.year == year] for year in range(2017, 2022)]

# Plotting total property and violent crimes
plt.figure(figsize=(12, 6), dpi=100)
sb.countplot(x="category_description", data=df).set_title(
    "Total Property & Violent Crimes (2003-Present)"
)
plt.xlabel("Crime Type")
plt.ylabel("Total Incidents")
plt.xticks(rotation=60)
plt.show()

# Displaying count of each crime category
display(df.category_description.value_counts())

# Plotting total violent crimes and murders by zip code
fig, axs = plt.subplots(figsize=(16, 6), ncols=2, dpi=100)
df_viol.zip_code.value_counts().head(25).plot.bar(
    ax=axs[0], title="Total Violent Crimes in Top 25 Zip Codes (2003-Present)", rot=60
)
df_mur.zip_code.value_counts().head(25).plot.bar(
    ax=axs[1], title="Total Murders in Top 25 Zip Codes (2003-Present)", rot=60
)
plt.show()

# Creating frequency tables for violent crimes and murders by zip code
viol_freq = pd.crosstab(df_viol.zip_code, df_viol.category_description)
mur_freq = pd.crosstab(df_mur.zip_code, df_mur.category_description)
display(viol_freq)

# Creating monthly dataframes for violent crimes
monthly_viol = pd.DataFrame(
    df_viol[df_viol["category_description"] == "Aggravated Assault"]
    .resample("M")
    .size()
)
monthly_viol.columns = ["Aggravated Assault"]

for crime_type in df_viol["category_description"].unique():
    monthly_viol[crime_type] = pd.DataFrame(
        df_viol[df_viol["category_description"] == crime_type].resample("M").size()
    )
monthly_viol["Total"] = monthly_viol.sum(axis=1)

# Plotting monthly trends for each type of violent crime
plt.figure(figsize=(16, 25), dpi=100)

i = 521
for crime_type in monthly_viol.columns:
    plt.subplot(i)
    monthly_viol[crime_type].plot(color="red", linewidth=1.5, label="Total per month")
    monthly_viol[crime_type].rolling(window=12).mean().plot(
        color="orange", linewidth=5, label="12-month Moving Average"
    )
    plt.title(crime_type, fontsize=12)
    plt.xlabel("")
    plt.legend(prop={"size": 12})
    plt.tick_params(labelsize=12)
    i += 1
viol_freq.to_csv("viol_freq.csv")

In [ ]:
display(monthly_viol.head())
display(viol_freq.head())

In [ ]:
viol_freq.to_csv("viol_freq.csv")

### Distribution of violent crime and murders across council districts, APD Districts, and APD sectors

In [ ]:
# Plotting violent crime distribution by council district
plt.figure(figsize=(12, 6), dpi=100)
pd.crosstab(df_viol.council_district, df_viol.category_description).plot.bar(
    rot=60,
    xlabel="Council District",
    ylabel="Crime Count",
    title="Violent Crime Distribution by Council District (2003-Present)",
)
plt.show()

# Plotting murder distribution by council district
plt.figure(figsize=(12, 6), dpi=100)
pd.crosstab(df_mur.council_district, df_mur.category_description).plot.bar(
    rot=60,
    xlabel="Council District",
    ylabel="Crime Count",
    title="Murder Distribution by Council District (2003-Present)",
    legend=False,
)
plt.show()

# Plotting violent crime distribution by APD sector
plt.figure(figsize=(12, 6), dpi=100)
pd.crosstab(df_viol.apd_sector, df_viol.category_description).plot.bar(
    rot=60,
    xlabel="APD Sector",
    ylabel="Crime Count",
    title="Violent Crime Distribution by APD Sector (2003-Present)",
)
plt.show()

# Plotting murder distribution by APD sector
plt.figure(figsize=(12, 6), dpi=100)
pd.crosstab(df_mur.apd_sector, df_mur.category_description).plot.bar(
    rot=60,
    xlabel="APD Sector",
    ylabel="Crime Count",
    title="Murder Distribution by APD Sector (2003-Present)",
    legend=False,
)
plt.show()

# Plotting violent crime distribution by APD district
plt.figure(figsize=(12, 6), dpi=100)
pd.crosstab(df_viol.apd_district, df_viol.category_description).plot.bar(
    rot=60,
    xlabel="APD District",
    ylabel="Crime Count",
    title="Violent Crime Distribution by APD District (2003-Present)",
)
plt.show()

# Plotting murder distribution by APD district
plt.figure(figsize=(12, 6), dpi=100)
pd.crosstab(df_mur.apd_district, df_mur.category_description).plot.bar(
    rot=60,
    xlabel="APD District",
    ylabel="Crime Count",
    title="Murder Distribution by APD District (2003-Present)",
    legend=False,
)
plt.show()

### Violent crime and murder distribution by location type

In [ ]:
# Calculating violent crime distribution by location type
viol_loc = pd.crosstab(df_viol.location_type, df_viol.category_description)
display(viol_loc)

# Calculating murder distribution by location type
mur_loc = pd.crosstab(df_mur.location_type, df_mur.category_description)

# Plotting violent crime distribution by location type
fig, axs = plt.subplots(figsize=(20, 14), dpi=100, ncols=2)
viol_loc.plot.barh(
    title="Violent Crime Distribution by Location Type (2003-Present)", ax=axs[0]
)
mur_loc.plot.barh(
    title="Murder Distribution by Location Type (2003-Present)", legend=False, ax=axs[1]
)
plt.show()

# Saving the violent crime distribution by location type to a CSV file
viol_loc.to_csv("viol_loc.csv")

### Clearance Analysis

In [ ]:
# Calculate the average time-to-clearance for all crimes
average_time_to_clearance = (df["clearance_date"] - df["report_date_time"]).mean()

# Convert the Timedelta to integer days
average_time_to_clearance_days = average_time_to_clearance.days

print(
    f"Average Time-to-Clearance for All Crimes: {average_time_to_clearance_days} days"
)

# Create a DataFrame for time-to-clearance by crime type
time_to_clearance_by_type = (
    df.groupby("highest_offense_description")["clearance_date"].mean()
    - df.groupby("highest_offense_description")["report_date_time"].mean()
)

# Convert the Timedelta to integer days
time_to_clearance_by_type_days = time_to_clearance_by_type.dt.days

# Plot the average time-to-clearance by crime type
plt.figure(figsize=(12, 6))
time_to_clearance_by_type_days.sort_values().plot(kind="barh")
plt.title("Average Time-to-Clearance by Crime Type")
plt.xlabel("Average Time-to-Clearance (Days)")
plt.ylabel("Crime Type")
plt.show()

# Calculate clearance rates by zip code
clearance_rates_by_zip = (
    df.groupby("zip_code")["clearance_status"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

# Plot the clearance rates for the top 10 zip codes
top_10_zip_codes = df["zip_code"].value_counts().head(10).index
clearance_rates_by_zip.loc[top_10_zip_codes].plot(kind="bar", stacked=True)
plt.title("Clearance Rates by Zip Code (Top 10)")
plt.xlabel("Zip Code")
plt.ylabel("Clearance Rate")
plt.legend(title="Clearance Status")
plt.show()


### How does violent crime appear on the map?

** Note: Rape incidents provide no location coordinates therefore cannot be shown on a map. **

In [ ]:
def create_heatmap(df, outfile):
    coords_heat = df[(df["latitude"].notnull()) & (df["longitude"].notnull())]

    map_austin = folium.Map(
        location=[30.2672, -97.7431], tiles="OpenStreetMap", zoom_start=12
    )
    map_austin.add_child(
        plugins.HeatMap(coords_heat[["latitude", "longitude"]].values, radius=15)
    )
    map_austin.save(outfile)


# Create heat map for Aggravated Assault
create_heatmap(df_agg_asslt, "agg_asslt_heatmap.html")

# Create heat map for Robbery
create_heatmap(df_robbery, "agg_robbery_heatmap.html")

# Create heat map for Murder
create_heatmap(df_mur, "mur_heatmap.html")

### Are there any addresses where violent crime and murder occurs frequently?

In [ ]:
# Show addresses with 50 or more reported violent crimes
df_viol_address_counts = df_viol.address.value_counts()
addresses_with_50_or_more_violent_crimes = df_viol_address_counts[df_viol_address_counts >= 50].to_frame()
addresses_with_50_or_more_violent_crimes

In [ ]:
# Show addresses with 2 or more reported murders
df_mur_address_counts = df_mur.address.value_counts()
addresses_with_2_or_more_murders = df_mur_address_counts[df_mur_address_counts >= 2].to_frame()
addresses_with_2_or_more_murders

In [ ]:
df_clean = df.copy()
df_clean.to_csv("df_clean.csv")

df_17.to_csv("df_17.csv")
df_18.to_csv("df_18.csv")
df_19.to_csv("df_19.csv")
df_20.to_csv("df_20.csv")
df_21.to_csv("df_21.csv")

df_viol_17.to_csv("df_viol_17.csv")
df_viol_18.to_csv("df_viol_18.csv")
df_viol_19.to_csv("df_viol_19.csv")
df_viol_20.to_csv("df_viol_20.csv")
df_viol_21.to_csv("df_viol_21.csv")

df_mur_17.to_csv("df_mur_17.csv")
df_mur_18.to_csv("df_mur_18.csv")
df_mur_19.to_csv("df_mur_19.csv")
df_mur_20.to_csv("df_mur_20.csv")
df_mur_21.to_csv("df_mur_21.csv")

df_viol.to_csv("df_viol.csv")
df_mur.to_csv("df_mur.csv")
df_agg_asslt.to_csv("df_agg_asslt.csv")
df_rape.to_csv("df_rape.csv")

In [ ]:
df_53 = df[df.zip_code == 78753]
df_05 = df[df.zip_code == 78705]
df_41 = df[df.zip_code == 78741]

df_01.to_csv("df_01.csv")
df_53.to_csv("df_53.csv")
df_41.to_csv("df_41.csv")
df_05.to_csv("df_05.csv")